In [1]:
""" Proof of concept for the random-cut-hyperplanes idea """
import sys
import numpy as np
from scipy.stats import scoreatpercentile
from sklearn.metrics import confusion_matrix

In [2]:
N_ESTIMATORS = 100
SCORE_AT = 2.5

# sys.setrecursionlimit(50000)

# Steps for the algorithm
#
# Given a dataset, X:
#   1. Get the feature ranges for each feature within X.
#   2. Sample a random point between each feature.
#   3. Create a hyperplane using each of these points.
#   4. Get the normal to that plane.
#   5. Determine which side of the plane the point lies on.
#   6. Repeat for the two sides.
#   7. If an individual point is left, stop iteration
#
class IsolationForest(object):
    def __init__(self, n_estimators=10):
        self.n_estimators = n_estimators

    def fit(self, points):
        self.trees = []
        for i in range(self.n_estimators):
            self.trees.append(IsolationTree(points))
        return self

    def decision_function(self, points):
        depths = np.array([tree.decision_function(points) for tree in self.trees])
        mean_depths = np.mean(depths, axis=0)
        # Normalize the points
        scores = 2**-(mean_depths / average_path_length(points.shape[0]))
        return scores

    def get_depths(self, points):
        depths = np.array([tree.decision_function(points) for tree in self.trees])
        mean_depths = np.mean(depths, axis=0)
        return mean_depths


class IsolationTree(object):
    def __init__(self, points, group_threshold=15, depth=1):
        self.child_left = self.child_right = None
        self.points = points
        self.group_threshold = group_threshold
        self.num_points = self.points.shape[0]
        self.split(self.points, depth)

    def __str__(self):
        return f"<{self.__class__.__name__} num_points:{self.num_points}>"

    @property
    def is_leaf(self):
        return self.child_left == None and self.child_right == None

    def split(self, points, depth=1):
        node, points_left, points_right = self.get_split(points)

        if not node or depth >= 50:
            return self

        self.node = node
        self.child_left = IsolationTree(points_left, depth=depth + 1)
        self.child_right = IsolationTree(points_right, depth=depth + 1)

    def decision_function(self, points):
        return np.array([self.get_depth(point) for point in points])

    def get_depth(self, point):
        if self.is_leaf:
            return 1
        else:
            if self.node.is_point_left(point):
                return 1 + self.child_left.get_depth(point)
            else:
                return 1 + self.child_right.get_depth(point)

    def get_split(self, points):
        if self.num_points <2:
            return (None, None, None)
        split_feature = sample_feature(points)
        split_threshold = sample_split_threshold(points, split_feature)
        node = Node(split_threshold, split_feature)

        positions = node.position_of_points(points)

        points_left = points[positions]
        points_right = points[np.logical_not(positions)]

        return (node, points_left, points_right)


class Node(object):
    def __init__(self, threshold, feature):
        self.threshold = threshold
        self.feature = feature

    def is_point_left(self, point):
        return point[self.feature] < self.threshold

    def position_of_points(self, points):
        return np.array([self.is_point_left(point) for point in points])


class RandomHyperplanes(object):
    def __init__(self, n_estimators=10):
        self.n_estimators = n_estimators

    def fit(self, points):
        self.planes = []
        for i in range(self.n_estimators): 
            self.planes.append(HyperplaneCollection(points))
        return self

    def decision_function(self, points):
        depths = np.array([plane.decision_function(points) for plane in self.planes])
        mean_depths = np.mean(depths, axis=0)
        # Normalize the points
        scores = 2**-(mean_depths / average_path_length(points.shape[0]))
        return scores

    def get_depths(self, points):
        depths = np.array([plane.decision_function(points) for plane in self.planes])
        mean_depths = np.mean(depths, axis=0)
        return mean_depths


class HyperplaneCollection(object):
    def __init__(self, points, group_threshold=15, depth=1):
        self.child_left = self.child_right = None
        self.points = points
        self.group_threshold = group_threshold
        self.num_points = self.points.shape[0]
        self.split(self.points, depth)

    def __str__(self):
        return f"<{self.__class__.__name__} num_points:{self.num_points}>"

    @property
    def is_leaf(self):
        return self.child_left == None and self.child_right == None

    def split(self, points, depth=1):
        plane, points_left, points_right = self.get_split(points)
        if not plane or depth >= 50:
            return self
        self.splitting_plane = plane
        self.child_left = HyperplaneCollection(points_left, depth=depth + 1)
        self.child_right = HyperplaneCollection(points_right, depth=depth + 1)

    def decision_function(self, points):
        return np.array([self.get_depth(point) for point in points])

    def get_depth(self, point):
        if self.is_leaf:
            return 1
        else:
            if self.splitting_plane.point_relative_to_plane(point) < 0:
                return 1 + self.child_left.get_depth(point)
            else:
                return 1 + self.child_right.get_depth(point)

    def get_split(self, points):
        if self.num_points < 2:
            return (None, None, None)
        
        splitting_plane = generate_splitting_plane(points)
        positions = splitting_plane.position_of_points(points)
        # split_point = np.random.uniform(np.min(positions), np.max(positions))
        
        points_left = points[np.where(positions < 0)]
        points_right = points[np.where(positions > 0)]
        return (splitting_plane, points_left, points_right)


class Hyperplane(object):
    def __init__(self, origin, normal):
        self.origin = origin
        self.normal = normal

    def point_relative_to_plane(self, point):
        return np.dot(self.normal, point - self.origin)

    def position_of_points(self, points):
        position = np.array([self.point_relative_to_plane(point) for point in points])
        return position


def sample_feature(point):
    length = point.shape[-1]
    return np.random.randint(low=0, high=length)

def sample_split_threshold(points, feature):
    return list(generate_point(points))[feature]

def average_path_length(n):
    return 2 * harmonic_approx(n - 1) - (2 * (n - 1) / n)

def harmonic_approx(n):
    return np.log(n) + 0.5772156649

def generate_splitting_plane(points):
    feature_ranges = get_feature_ranges(points)
    origin = np.fromiter(generate_point(points), dtype=float)
    
    normal = np.fromiter(generate_point(points), dtype=float)
    normal -= origin
    
    return Hyperplane(origin=origin, normal=normal)
    
def generate_point(points):
    """ Generat an n-dimensional normal vector

    For now just do so by sampling the points from a uniform distribution.
    """
    feature_mins_maxes = get_feature_ranges(points)
    for min_, max_, in get_feature_ranges(points):
        yield np.random.uniform(low=min_, high=max_)

def get_feature_ranges(points):
    """ Yields the min and max of each feature in a vector. """
    points_transpose = np.transpose(points)
    for point in points_transpose:
        yield (np.min(point), np.max(point))

In [3]:
def _gen_hard_data(n, p, infection_pct, variance=10.0, mu=5.0):
    X = np.random.randn(n, p)

    # hard data
    # Weight it to the number of features
    is_anomaly = np.random.rand(n, p) < (infection_pct / p)
    X[is_anomaly] = variance * np.random.randn() + mu

    y = np.zeros(shape=(n,))

    tmp = np.array([np.any(r) for r in is_anomaly])
    y[tmp] = 1.0

    return (X, y)

def _gen_easy_data(n, p, infection_pct, variance=10.0, mu=5.0):
    X = np.random.randn(n, p)
    is_anomaly = np.random.choice(n, size=int(infection_pct*n), replace=False)
    X[is_anomaly] = variance * np.random.randn(is_anomaly.shape[0], p) + mu
    y = np.zeros(shape=(n,))
    y[is_anomaly] = 1.0
    return (X, y)

In [4]:
def run_plane_simul(points, y):
    print("Beginning plane fit...")
    rhp = RandomHyperplanes(n_estimators=N_ESTIMATORS)
    rhp = rhp.fit(points)
    print("done fitting")

    scores = rhp.decision_function(points)
    threshold = scoreatpercentile(scores, 100 - SCORE_AT)
    anomalies = scores >= threshold
    y_pred = np.zeros(shape=anomalies.shape)
    y_pred[anomalies] = 1
    
    """
    correct_guesses = np.count_nonzero(y[np.where(scores <= threshold)])
    incorrect_guesses = y[np.where(scores <= threshold)].shape[0] - \
        correct_guesses

    print("Correct guesses:", correct_guesses)
    print("Incorrect guesses:", incorrect_guesses)
    print("Expected", np.count_nonzero(y), "anomalies")
    """
    cnf_matrix = confusion_matrix(y, y_pred)

    """
    tn, fp, fn, tp = cnf_matrix.ravel()
    print(f"tp: {tp} \ntn: {tn} \nfp: {fp} \nfn: {fn}")
    """
    cnf_matrix = cnf_matrix.astype('float') / \
        cnf_matrix.sum(axis=1)[:, np.newaxis]

    """
    tn, fp, fn, tp = cnf_matrix.ravel()
    print(f"Normalized \ntp: {tp} \ntn: {tn} \nfp: {fp} \nfn: {fn}")
    """
    print(cnf_matrix)

    depths = rhp.get_depths(points)
    anomalous_depths = depths[np.where(y==1.0)]
    print("Average anomalous depth:", np.mean(anomalous_depths))

    non_anomalous_depths = depths[np.where(y==0.0)]
    print("Average non-anomalous depth:", np.mean(non_anomalous_depths))
    return (scores, depths, y_pred, y)


def run_iforest_simul(points, y):
    print("Beginning iforest fit...")
    iforest = IsolationForest(n_estimators=N_ESTIMATORS)
    iforest = iforest.fit(points)
    print("done fitting")

    scores = iforest.decision_function(points)
    threshold = scoreatpercentile(scores, 100 - SCORE_AT)
    anomalies = scores >= threshold
    y_pred = np.zeros(shape=anomalies.shape)
    y_pred[anomalies] = 1

    """
    correct_guesses = np.count_nonzero(y[np.where(scores <= threshold)])
    incorrect_guesses = y[np.where(scores <= threshold)].shape[0] - \
        correct_guesses

    print("iforest Correct guesses:", correct_guesses)
    print("iforest Incorrect guesses:", incorrect_guesses)
    print("Expected", np.count_nonzero(y), "anomalies")
    """
    iforest_cnf_matrix = confusion_matrix(y, y_pred)
    """
    tn, fp, fn, tp = iforest_cnf_matrix.ravel()
    print(f"tp: {tp} \ntn: {tn} \nfp: {fp} \nfn: {fn}")
    """
    iforest_cnf_matrix = iforest_cnf_matrix.astype('float') / \
            iforest_cnf_matrix.sum(axis=1)[:, np.newaxis]

    print(iforest_cnf_matrix)

    """
    tn, fp, fn, tp = iforest_cnf_matrix.ravel()
    print(f"Normalized \ntp: {tp} \ntn: {tn} \nfp: {fp} \nfn: {fn}")
    """
    depths = iforest.get_depths(points)
    anomalous_depths = depths[np.where(y==1.0)]
    print("Average anomalous depth:", np.mean(anomalous_depths))

    non_anomalous_depths = depths[np.where(y==0.0)]
    print("Average non-anomalous depth:", np.mean(non_anomalous_depths))
    return (scores, depths, y_pred, y)

In [5]:
n = 1000 # number of entries
p = 2    # features

infection_pct = 0.05
X, y = _gen_easy_data(n, p, infection_pct)

scores_r, depths_r, y_pred_r, y_r = run_plane_simul(X, y)
print("\nDone plane simul-----\n")
scores_i, depths_i, y_pred_i, y_i = run_iforest_simul(X, y)

Beginning plane fit...
done fitting
[[ 1.   0. ]
 [ 0.5  0.5]]
Average anomalous depth: 10.88
Average non-anomalous depth: 25.6531052632

Done plane simul-----

Beginning iforest fit...
done fitting
[[ 1.   0. ]
 [ 0.5  0.5]]
Average anomalous depth: 9.368
Average non-anomalous depth: 20.3250842105


In [6]:
X[np.where(y==1.0)]

array([[  7.57465909e+00,   8.56434098e+00],
       [  5.68247247e+00,  -4.04386367e+00],
       [  3.29215027e-01,   1.29935858e+01],
       [  4.21284127e+00,   4.41438584e+00],
       [  6.26269186e+00,  -1.38089816e+00],
       [  1.92749598e+01,   3.66927921e+00],
       [ -1.00043148e+00,  -9.14574224e+00],
       [  1.58880489e+01,   2.39339553e+01],
       [  4.27565851e+00,   1.69675957e+01],
       [ -5.50948703e+00,  -8.79222198e-01],
       [  5.36613107e+00,   2.66032836e-01],
       [  5.27617031e+00,   1.34412185e+01],
       [  1.47605715e+00,   8.37471240e-01],
       [  2.86397056e+01,   2.58300135e+01],
       [  5.01329910e+00,   1.62176145e+01],
       [  1.50639108e+01,  -1.69728966e+00],
       [  1.66043824e+01,   3.81643983e+00],
       [  6.29731429e+00,   5.88248035e+00],
       [  6.43350507e+00,   1.82696068e+01],
       [  1.72926924e+01,  -1.44294323e+01],
       [  5.87453366e+00,   2.61395326e+00],
       [  5.34197127e+00,  -4.98685654e+00],
       [  

In [7]:
rhp = RandomHyperplanes(n_estimators=N_ESTIMATORS)
rhp = rhp.fit(X)
depths = rhp.get_depths(X)

In [8]:
print(np.unique(depths))

[  4.79   6.41   6.64   6.77   7.08   7.14   7.15   7.34   7.99   8.05
   8.25   8.37   8.5    8.76   9.03   9.46   9.47   9.58   9.64   9.89
  10.35  10.43  10.5   11.23  11.24  11.25  11.99  12.08  12.14  12.23
  12.42  12.45  12.59  12.66  12.76  12.78  12.9   13.03  13.11  13.17
  13.23  13.34  13.93  13.94  14.01  14.13  14.23  14.31  14.61  14.67
  14.88  14.92  15.44  15.47  15.72  15.8   15.93  16.    16.04  16.09
  16.35  16.46  16.62  16.67  17.05  17.19  17.28  17.38  17.52  17.6
  17.79  17.81  17.92  18.01  18.04  18.08  18.2   18.21  18.25  18.48
  18.67  18.74  18.77  18.93  19.    19.01  19.07  19.12  19.13  19.14
  19.2   19.22  19.25  19.31  19.37  19.38  19.56  19.7   19.75  19.79
  19.8   19.87  19.95  19.99  20.03  20.06  20.13  20.14  20.17  20.2
  20.28  20.29  20.35  20.38  20.42  20.49  20.54  20.55  20.56  20.58
  20.63  20.66  20.72  20.76  20.85  20.94  20.96  20.97  20.98  21.    21.1
  21.18  21.2   21.22  21.24  21.27  21.29  21.35  21.37  21.38  21.39
  